In [130]:
import cdsapi
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [131]:
floods = pd.read_csv('../data/Flood Events/India_Floods_Inventory.csv')


In [132]:
floods.head()

,UEI,Start Date,End Date,Duration(Days),Main Cause,Location,Districts,State,Latitude,Longitude,Severity,Area Affected,Human fatality,Human injured,Human Displaced,Animal Fatality,Description of Casualties/injured,Extent of damage,Event Source,Event Souce ID
0,UEI-IMD-FL-2015-0001,2015-06-20,2015-06-21,1,Heavy rains,NaN,"East Godavari, Srikakulam, Visakhapatnam and W...",ANDHRA PRADESH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,i) Damage to 2000 hectares of crops reported. ...,IMD,NaN
1,UEI-IMD-FL-2015-0002,2015-11-15,2015-11-23,8,Heavy rains,NaN,"Anantapur, Chittoor, East Godavari, Krishna, N...",ANDHRA PRADESH,NaN,NaN,NaN,NaN,88.0,NaN,NaN,16710.0,88 persons died. 16710 animals perished. (844 ...,i) Extensive damage to Agricultural crops (mor...,IMD,NaN
2,UEI-IMD-FL-2015-0003,2015-12-22,2015-12-22,0,Heavy rains,NaN,Vishakhapatnam,ANDHRA PRADESH,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4 persons died due to landslips,Landslips,IMD,NaN
3,UEI-IMD-FL-2015-0004,2015-10-06,2015-10-06,0,Heavy rains,NaN,Parts of Arunachal Pradesh,ARUNACHAL PRADESH,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2 persons died.,NaN,IMD,NaN
4,UEI-IMD-FL-2015-0005,2015-02-19,2015-02-19,0,Heavy rains,NaN,Parts of Assam,ASSAM,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2 persons died due to landslide,NaN,IMD,NaN


In [133]:
floods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1027 entries, 0 to 1026
Data columns (total 20 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   UEI                                1027 non-null   object 
 1   Start Date                         1027 non-null   object 
 2   End Date                           1027 non-null   object 
 3   Duration(Days)                     1027 non-null   object 
 4   Main Cause                         905 non-null    object 
 5   Location                           276 non-null    object 
 6   Districts                          464 non-null    object 
 7   State                              464 non-null    object 
 8   Latitude                           325 non-null    float64
 9   Longitude                          325 non-null    float64
 10  Severity                           262 non-null    float64
 11  Area Affected                      262 non-null    float

In [134]:
# check which events are not rains or floods, if there is any issue with them
# probably safe to drop the main cause column
floods[~floods['Main Cause'].str.contains("rain|flood", case=False, na=False)]['Main Cause'].value_counts()

Main Cause
Tropical cyclone              16
Landslide                      8
Dam/Levy, break or release     8
Cloud Burst                    6
Massive landslide              1
Moderate                       1
Tropical Cyclone               1
Extra-tropical cyclone         1
Name: count, dtype: int64

In [135]:
# where data came from, not needed
floods['UEI'].unique()
floods['Event Souce ID'].unique()
floods['Event Source'].unique()

array(['IMD', 'DFO', 'EM-DAT'], dtype=object)

In [136]:
# There are 276 locations where we can derive which state the flood was in
mask = (~floods['Location'].isnull() 
       & floods['Latitude'].isnull()
       & floods['Longitude'].isnull()
       & floods['State'].isnull())

floods[mask]['Location'].count()
floods[mask].head()

,UEI,Start Date,End Date,Duration(Days),Main Cause,Location,Districts,State,Latitude,Longitude,Severity,Area Affected,Human fatality,Human injured,Human Displaced,Animal Fatality,Description of Casualties/injured,Extent of damage,Event Source,Event Souce ID
726,UEI-EM-DAT-FL-1926-0001,1926-07-19,1926-07-19,0,Riverine flood,Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EM-DAT,1926-0008
727,UEI-EM-DAT-FL-1928-0001,1928-10-19,1928-10-19,0,NaN,"Godavari, Vizagapatnam",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EM-DAT,1928-0018
728,UEI-EM-DAT-FL-1950-0001,1950-07-11,1950-07-11,0,Riverine flood,Saurashtra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EM-DAT,1950-0006
731,UEI-EM-DAT-FL-1956-0001,1956-08-01,1956-08-01,0,NaN,"Gujarat, Saurashtra, Kutch",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EM-DAT,1956-0012
732,UEI-EM-DAT-FL-1956-0002,1956-09-09,1956-09-09,0,NaN,"Bihar, Uttar Pradesh",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EM-DAT,1956-0016


In [137]:
# There are no districts where we can derive which state the flood was in
floods[~floods['Districts'].isnull() & floods['State'].isnull()]['Location'].count()

0

In [138]:
# We can use 63 co-ordinate pairs to derive which state the flood was in
mask = (~floods['Latitude'].isnull() & 
       floods['Location'].isnull()  
       & floods['State'].isnull())

floods[mask]['Latitude'].count()
floods[mask].head()

,UEI,Start Date,End Date,Duration(Days),Main Cause,Location,Districts,State,Latitude,Longitude,Severity,Area Affected,Human fatality,Human injured,Human Displaced,Animal Fatality,Description of Casualties/injured,Extent of damage,Event Source,Event Souce ID
464,UEI-DFO-FL-1985-0001,1985-06-23,1985-06-27,4,Heavy rain,NaN,NaN,NaN,14.5398,75.0937,1.0,254234.60,95.0,NaN,25000.0,NaN,NaN,NaN,DFO,27
465,UEI-DFO-FL-1985-0002,1985-07-18,1985-07-30,12,Heavy rain,NaN,NaN,NaN,32.8353,76.9103,1.0,117441.17,340.0,NaN,20000.0,NaN,NaN,NaN,DFO,33
466,UEI-DFO-FL-1985-0003,1985-09-13,1985-10-15,32,Heavy rain,NaN,NaN,NaN,26.4816,82.8434,2.0,89994.67,557.0,NaN,152000.0,NaN,NaN,NaN,DFO,53
467,UEI-DFO-FL-1985-0004,1985-10-18,1985-10-20,2,Tropical cyclone,NaN,NaN,NaN,21.1495,86.7154,2.0,46758.64,49.0,NaN,150000.0,NaN,NaN,NaN,DFO,61
468,UEI-DFO-FL-1986-0001,1986-06-15,1986-08-20,66,Monsoonal rain,NaN,NaN,NaN,25.6339,84.0726,1.0,507167.44,258.0,NaN,3040000.0,NaN,NaN,NaN,DFO,93


In [139]:
# ranking of severity check, 
# maybe useful to classify severity of flood based on amount of rainfall over a period of time
floods['Severity'].describe()

count    262.000000
mean       1.232824
std        0.357180
min        1.000000
25%        1.000000
50%        1.000000
75%        1.500000
max        2.000000
Name: Severity, dtype: float64

In [140]:
# now drop irrelevant columns
floods.isnull().sum()

UEI                                    0
Start Date                             0
End Date                               0
Duration(Days)                         0
Main Cause                           122
Location                             751
Districts                            563
State                                563
Latitude                             702
Longitude                            702
Severity                             765
Area Affected                        765
Human fatality                       422
Human injured                        943
Human Displaced                      765
Animal Fatality                      990
Description of Casualties/injured    644
Extent of damage                     693
Event Source                           0
Event Souce ID                       464
dtype: int64

In [141]:
floods_cleaned = floods.drop([
    'UEI',
    'Animal Fatality', 
    'Description of Casualties/injured', 
    'Extent of damage ',
    'Districts',
    'Area Affected',
    'Human injured',
    'Human fatality',
    'Human Displaced',
    'Main Cause',
    'Event Source',
    'Event Souce ID'
    ],axis=1)


In [142]:
floods_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1027 entries, 0 to 1026
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Start Date      1027 non-null   object 
 1   End Date        1027 non-null   object 
 2   Duration(Days)  1027 non-null   object 
 3   Location        276 non-null    object 
 4   State           464 non-null    object 
 5   Latitude        325 non-null    float64
 6   Longitude       325 non-null    float64
 7   Severity        262 non-null    float64
dtypes: float64(3), object(5)
memory usage: 64.3+ KB


# Fill Nulls

In [143]:
floods_cleaned.isnull().sum()

Start Date          0
End Date            0
Duration(Days)      0
Location          751
State             563
Latitude          702
Longitude         702
Severity          765
dtype: int64

In [153]:
floods_cleaned['State'].isnull().sum()

563

In [150]:
# locate location rows where state name is in location
# if the states word is in location and the state is null
# fill the null value with the correct state

df_floods = floods_cleaned.copy()

states = df_floods['State'].dropna().unique()
df_floods['Location'] = df_floods['Location'].fillna("None")

def fill_state(row):
    if pd.isna(row["State"]):
        for state in states:
            if state.lower().strip() in row["Location"].lower().strip():
                return state
    return row["State"]


df_floods['State'] = df_floods.apply(fill_state, axis=1)

In [ ]:
df_floods['State'].isnull().sum()

333

In [159]:
states

array(['ANDHRA  PRADESH', 'ARUNACHAL PRADESH', 'ASSAM', 'BIHAR',
       'CHATTISGARH', 'HIMACHAL PRADESH', 'JAMMU & KASHMIR', 'KARNATAKA',
       'KERALA', 'MADHYA PRADESH', 'MAHARASHTRA', 'MEGHALAYA', 'ODISHA',
       'PUNJAB', 'RAJASTHAN', 'SIKKIM', 'TAMIL NADU', 'TELANGANA',
       'UTTAR PRADESH', 'UTTARAKHAND', 'WEST BENGAL', 'PUDUCHERRY',
       'ANDHRA PRADESH', 'CHHATTISGARH', 'GUJARAT', 'JHARKHAND',
       'MIZORAM', 'TRIPURA', 'NAGALAND', 'ANDAMAN & NICOBAR ISLANDS',
       'NEW DELHI'], dtype=object)

In [168]:
# get state based on latitude and longitude
from geopy.geocoders import Nominatim
import time

df_floods_2 = df_floods.copy()

geolocator = Nominatim(user_agent="state_filler")

def get_state_from_coords(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language="en")
        for state in states:
            if state.lower().strip() in str(location).lower().strip():
                return state
    except Exception as e:
        return None

# Fill missing states
for idx, row in df_floods_2.iterrows():
    if pd.isna(row["State"]):
        state = get_state_from_coords(row["Latitude"], row["Longitude"])
        df_floods_2.at[idx, "State"] = state
        time.sleep(1) 

print(df_floods_2)

      Start Date    End Date Duration(Days)  \
0     2015-06-20  2015-06-21              1   
1     2015-11-15  2015-11-23              8   
2     2015-12-22  2015-12-22              0   
3     2015-10-06  2015-10-06              0   
4     2015-02-19  2015-02-19              0   
...          ...         ...            ...   
1022  2019-06-28  2019-07-04              6   
1023  2019-07-14  2019-09-30             78   
1024  2019-08-12  2019-08-15              3   
1025  2019-10-19  2019-10-23              4   
1026  2019-11-29  2019-12-04              5   

                                               Location              State  \
0                                                  None    ANDHRA  PRADESH   
1                                                  None    ANDHRA  PRADESH   
2                                                  None    ANDHRA  PRADESH   
3                                                  None  ARUNACHAL PRADESH   
4                                            

In [169]:
df_floods_2['State'].isnull().sum()

86

In [176]:
df_floods_3 = df_floods_2.copy()
df_floods_3 = df_floods_3.drop(['Location','Latitude', 'Longitude', 'Severity'], axis=1)

In [ ]:
df_floods_3 = df_floods_3.sort_values(by='Start Date')

In [1]:
df_floods_3.to_csv("saved_states.csv")

NameError: name 'df_floods_3' is not defined